In [ ]:
!pip install python-telegram-bot requests Groq nest-asyncio

In [ ]:
import sqlite3
import os
import logging
from datetime import datetime
import json
import random
import requests
import asyncio
from telegram import Update
import nest_asyncio
from telegram.ext import Application, ContextTypes, MessageHandler, filters

# Настройка логирования
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)
logger = logging.getLogger(__name__)

# Конфигурация
TOKEN = ""
GROQ_API_KEY = ""
DB_NAME = "chat_bot.db"
GROQ_API_URL = "https://api.groq.com/openai/v1/chat/completions"

# Инициализация базы данных
def init_db():
    try:
        # Удаляем старую базу, если она повреждена
        if os.path.exists(DB_NAME):
            os.remove(DB_NAME)
            logger.info("Удалена поврежденная база данных")

        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()

        cursor.execute('''
        CREATE TABLE IF NOT EXISTS chat_interactions (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp TEXT NOT NULL,
            chat_id TEXT NOT NULL,
            chat_title TEXT,
            context_messages TEXT NOT NULL,
            detected_topic TEXT,
            sentiment TEXT,
            bot_response TEXT,
            response_generated INTEGER NOT NULL,
            participants_count INTEGER
        )
        ''')

        conn.commit()
        logger.info("База данных успешно инициализирована")
    except Exception as e:
        logger.error(f"Ошибка инициализации базы данных: {e}")
        raise
    finally:
        if conn:
            conn.close()

init_db()

def save_interaction_to_db(data):
    conn = None
    try:
        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()

        cursor.execute('''
        INSERT INTO chat_interactions (
            timestamp, chat_id, chat_title, context_messages,
            detected_topic, sentiment, bot_response,
            response_generated, participants_count
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            data['timestamp'],
            data['chat_id'],
            data.get('chat_title'),
            json.dumps(data['context_messages'], ensure_ascii=False),
            data.get('detected_topic'),
            data.get('sentiment'),
            data.get('bot_response'),
            int(data['response_generated']),
            data.get('participants_count')
        ))

        conn.commit()
    except Exception as e:
        logger.error(f"Ошибка сохранения в базу данных: {e}")
    finally:
        if conn:
            conn.close()

async def analyze_context_with_llama(messages):
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }

    # Формируем промпт с правильным форматированием
    prompt_messages = "\n".join(messages[-5:])  # Сначала объединяем сообщения

    prompt = f"""
    Проанализируйте следующие сообщения из чата и ответьте в формате JSON:
    1. Определите основную тему разговора (detected_topic)
    2. Определите тон беседы (sentiment: позитивный/нейтральный/негативный)
    3. Сгенерируйте уместный ответ (bot_response) или null, если ответ не требуется

    Сообщения:
    {prompt_messages}

    Ответ в формате:
    {{
        "detected_topic": "тема",
        "sentiment": "тон",
        "bot_response": "ответ или null"
    }}
    """

    payload = {
        "model": "llama-3.3-70b-versatile",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.1,
        "response_format": {"type": "json_object"}
    }

    try:
        response = requests.post(GROQ_API_URL, headers=headers, json=payload)
        response.raise_for_status()
        result = response.json()

        if 'choices' in result and len(result['choices']) > 0:
            content = result['choices'][0]['message']['content']
            return json.loads(content)

        return {
            "detected_topic": None,
            "sentiment": "нейтральный",
            "bot_response": None
        }
    except Exception as e:
        logger.error(f"Ошибка при анализе контекста через Groq: {e}")
        return {
            "detected_topic": None,
            "sentiment": "нейтральный",
            "bot_response": None
        }

async def handle_group_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    if update.message.chat.type not in ["group", "supergroup"]:
        return

    if 'recent_messages' not in context.chat_data:
        context.chat_data['recent_messages'] = []

    # Добавляем текущее сообщение в историю
    context.chat_data['recent_messages'].append(update.message)

    # Сохраняем только последние 5 сообщений
    context.chat_data['recent_messages'] = context.chat_data['recent_messages'][-5:]

    # Формируем список сообщений для анализа
    messages = [
        f"{msg.from_user.first_name}: {msg.text}"
        for msg in context.chat_data['recent_messages']
        if msg.text
    ]

    if not messages:
        return

    analysis = await analyze_context_with_llama(messages)

    interaction_data = {
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "chat_id": str(update.message.chat.id),
        "chat_title": update.message.chat.title,
        "context_messages": messages,
        "detected_topic": analysis.get("detected_topic"),
        "sentiment": analysis.get("sentiment"),
        "bot_response": analysis.get("bot_response"),
        "response_generated": analysis.get("bot_response") is not None,
        "participants_count": len({msg.from_user.id for msg in context.chat_data['recent_messages']})
    }

    save_interaction_to_db(interaction_data)

    if analysis.get("bot_response") and len(messages) > 3 and random.random() < 0.2:
        await update.message.reply_text(analysis["bot_response"])

# Функция запуска бота (должна быть определена перед вызовом)
async def run_bot():
    application = Application.builder().token(TOKEN).build()
    application.add_handler(MessageHandler(filters.ChatType.GROUPS, handle_group_message))

    print("Бот запущен. Нажмите Stop в Colab для остановки")
    await application.run_polling()

# Основной блок выполнения
if __name__ == "__main__":
    # Запускаем бот
    loop = asyncio.get_event_loop()
    try:
        loop.run_until_complete(run_bot())
    except KeyboardInterrupt:
        print("Бот остановлен")
    finally:
        loop.close()

In [ ]:
import sqlite3
conn = sqlite3.connect('chat_bot.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM chat_interactions")
print(cursor.fetchall())
conn.close()